In [78]:
import os
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from sklearn.feature_selection import SelectPercentile
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
from scipy.stats import randint
import joblib
import logging
from datetime import datetime
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.ensemble import RandomForestRegressor
from sklearn.feature_selection import SelectPercentile
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import RandomizedSearchCV
from itertools import product


In [79]:

# Helper function for trimming the dataset
def trim_set(frame: pd.DataFrame) -> pd.DataFrame:
    return frame[((frame['TRB'] > 2) | (frame['AST'] > 5)) & (frame['PTS'] > 15)]

# Function to get the NBA season year
def get_nba_season_year(date=None):
    if date is None:
        date = datetime.now()
    current_month = date.month
    current_year = date.year
    nba_season_year = current_year if current_month < 10 else current_year + 1
    return nba_season_year

In [80]:
current_season_start_year = get_nba_season_year() - 1
mvp_data_dir = '../Basketball Reference Stat Scraper/Awards/MVP'
player_data_dir = '../Basketball Reference Stat Scraper/player_stats/'

full_data = [
    pd.merge(
        pd.read_csv(os.path.join(player_data_dir, f'{year}_player_stats.csv'), index_col=0).assign(Player=lambda df: df.Player.str.strip('*')),
        pd.read_csv(os.path.join(mvp_data_dir, f'{year}_MVP.csv'), index_col=0),
        how='left'
    ).fillna(0)
    for year in range(1980, current_season_start_year + 1)
]

train = pd.concat(full_data)
df = trim_set(train)

feature_list = ['Age', 'G', 'GS', 'MP', 'FG', 'FGA', 'FG%', '3P', '3PA', '3P%', '2P', '2PA', '2P%', 'eFG%', 'FT', 'FTA',
                'FT%', 'ORB', 'DRB', 'TRB', 'AST', 'STL', 'BLK', 'TOV', 'PF', 'PTS', 'PER', 'TS%', '3PAr', 'FTr',
                'ORB%', 'DRB%', 'TRB%', 'AST%', 'STL%', 'BLK%', 'TOV%', 'USG%', 'OWS', 'DWS', 'WS', 'WS/48', 'OBPM',
                'DBPM', 'BPM', 'VORP']

x = df[feature_list]
y = df['Share']

In [88]:
from sklearn.model_selection import BaseCrossValidator

class CustomCrossValidator(BaseCrossValidator):
    def __init__(self, trb_threshold=2, ast_threshold=5, pts_threshold=15):
        self.trb_threshold = trb_threshold
        self.ast_threshold = ast_threshold
        self.pts_threshold = pts_threshold

    def split(self, X, y=None, groups=None):
        # Apply the trimming logic to determine the indices to keep
        keep_indices = ((X['TRB'] > self.trb_threshold) | 
                        (X['AST'] > self.ast_threshold)) & \
                        (X['PTS'] > self.pts_threshold)
        
        # Generate train/test indices using the filtered indices
        # You can use KFold, StratifiedKFold, etc., depending on your requirement
        for train_idx, test_idx in KFold(n_splits=5).split(X[keep_indices]):
            yield train_idx, test_idx

    def get_n_splits(self, X, y=None, groups=None):
        return 5  # or the number of splits you want


In [81]:
from itertools import product

param_grid = {
    'trb_threshold': trb_values,
    'ast_threshold': ast_values,
    'pts_threshold': pts_values
}

# Generate all possible combinations
param_combinations = list(product(trb_values, ast_values, pts_values))
len(param_combinations)

1210

In [84]:

from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.ensemble import RandomForestRegressor
from sklearn.feature_selection import SelectPercentile
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import RandomizedSearchCV


class DataTrimmer(BaseEstimator, TransformerMixin):
    def __init__(self, trb_index=0, ast_index=1, pts_index=2, trb_threshold=2, ast_threshold=5, pts_threshold=15):
        self.trb_index = trb_index
        self.ast_index = ast_index
        self.pts_index = pts_index
        self.trb_threshold = trb_threshold
        self.ast_threshold = ast_threshold
        self.pts_threshold = pts_threshold

    def fit(self, X, y=None):
        print("Shape of X:", X.shape)
        print("Shape of y:", y.shape)

        return self

    def transform(self, X, y=None):
        # Correct the logical expression with line continuation
        trimmed_indices = ((X[:, self.trb_index] > self.trb_threshold) | 
                           (X[:, self.ast_index] > self.ast_threshold)) & \
                           (X[:, self.pts_index] > self.pts_threshold)

        trimmed_data = X[trimmed_indices]

        if y is not None:
            trimmed_target = y[trimmed_indices]
            return trimmed_data, trimmed_target

        return trimmed_data




In [89]:
from sklearn.model_selection import GridSearchCV

# Define your pipeline without the DataTrimmer
pipeline = Pipeline([
    ('scaler', MinMaxScaler()),
    ('model', RandomForestRegressor(n_jobs=-1))
])

# Define the parameter grid, including trb, ast, and pts thresholds
param_grid = {
    'trb_threshold': [2, 3, 4],  # Example values
    'ast_threshold': [5, 6, 7],
    'pts_threshold': [15, 16, 17],
    'model__n_estimators': [100, 200],
    'model__max_depth': [10, 20]
}

# Create GridSearchCV with the custom cross-validator
grid_search = RandomizedSearchCV(
    estimator=pipeline,
    param_grid=param_grid,
    cv=CustomCrossValidator(),  # Using the custom cross-validator
    verbose=3,
    n_jobs=-1
)

# Fit the grid search
random_search.fit(X, y)



TypeError: RandomizedSearchCV.__init__() got an unexpected keyword argument 'param_grid'

In [86]:
# Define a simplified pipeline for debugging
simple_pipeline = Pipeline([
    ('data_trimmer', DataTrimmer(trb_index=0, ast_index=1, pts_index=2)),  # Adjust indices as needed
    ('model', RandomForestRegressor(n_jobs=-1))
])

# Fit the simple pipeline with x and y
simple_pipeline.fit(x, y)


Shape of X: (2876, 46)
Shape of y: (2876,)


ValueError: Found input variables with inconsistent numbers of samples: [2639, 2876]

[CV 4/5] END model__max_depth=80, model__max_features=sqrt, model__min_samples_leaf=3, model__min_samples_split=4, model__n_estimators=400, preprocessor__data_trimmer__ast_threshold=6, preprocessor__data_trimmer__pts_threshold=8, preprocessor__data_trimmer__trb_threshold=4;, score=nan total time=   0.0s
[CV 3/5] END model__max_depth=30, model__max_features=log2, model__min_samples_leaf=4, model__min_samples_split=10, model__n_estimators=200, preprocessor__data_trimmer__ast_threshold=7, preprocessor__data_trimmer__pts_threshold=10, preprocessor__data_trimmer__trb_threshold=4;, score=nan total time=   0.0s
[CV 3/5] END model__max_depth=80, model__max_features=None, model__min_samples_leaf=9, model__min_samples_split=2, model__n_estimators=400, preprocessor__data_trimmer__ast_threshold=9, preprocessor__data_trimmer__pts_threshold=16, preprocessor__data_trimmer__trb_threshold=4;, score=nan total time=   0.0s
[CV 4/5] END model__max_depth=20, model__max_features=None, model__min_samples_lea

In [ ]:


# Define the preprocessor pipeline
preprocessor = Pipeline([
    ('data_trimmer', DataTrimmer()),  # Use default threshold values
    ('feat_selection', SelectPercentile(percentile=60)),
    ('scaler', MinMaxScaler())
])

# Define the full pipeline with RandomForestRegressor
pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('model', RandomForestRegressor(n_jobs=-1))
])



param_grid = {
    'preprocessor__data_trimmer__trb_threshold': range(0,11,1),
    'preprocessor__data_trimmer__ast_threshold': range(0,11,1),
    'preprocessor__data_trimmer__pts_threshold': range(0,20,2),
    'model__n_estimators': range(100, 501, 100),
    'model__max_depth': range(10, 101, 10),
    'model__min_samples_split': range(2, 11, 2),
    'model__min_samples_leaf': range(1, 11, 1),
    'model__max_features': ['sqrt', 'log2', None] 
}

# Create RandomizedSearchCV
random_search = RandomizedSearchCV(
    pipeline, 
    param_distributions=param_grid, 
    n_iter=100, 
    cv=5, 
    verbose=3, 
    random_state=42, 
    n_jobs=-1
)




In [67]:
# assert len(x) == len(y)

# # Convert x and y to NumPy arrays
# x = x.values
# y = y.values


In [68]:
random_search.fit(x, y)


Fitting 5 folds for each of 100 candidates, totalling 500 fits


ValueError: 
All the 500 fits failed.
It is very likely that your model is misconfigured.
You can try to debug the error by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
500 fits failed with the following error:
Traceback (most recent call last):
  File "/Users/chaseallbright/opt/anaconda3/envs/NBA/lib/python3.10/site-packages/sklearn/model_selection/_validation.py", line 732, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/Users/chaseallbright/opt/anaconda3/envs/NBA/lib/python3.10/site-packages/sklearn/base.py", line 1151, in wrapper
    return fit_method(estimator, *args, **kwargs)
  File "/Users/chaseallbright/opt/anaconda3/envs/NBA/lib/python3.10/site-packages/sklearn/pipeline.py", line 416, in fit
    Xt = self._fit(X, y, **fit_params_steps)
  File "/Users/chaseallbright/opt/anaconda3/envs/NBA/lib/python3.10/site-packages/sklearn/pipeline.py", li

In [17]:
best_params = random_search.best_params_
best_score = random_search.best_score_
print(f"Best parameters: {best_params}")
print(f"Best score: {best_score}")


Best parameters: {'model__n_estimators': 200, 'model__min_samples_split': 6, 'model__min_samples_leaf': 3, 'model__max_features': 'sqrt', 'model__max_depth': 10}
Best score: 0.6260454322062774


In [19]:


# Create a parameter grid for Grid Search based on best_params
param_grid = {
    'model__n_estimators': [best_params['model__n_estimators'] - 10, best_params['model__n_estimators'], best_params['model__n_estimators'] + 10],
    'model__max_depth': [best_params['model__max_depth'] - 2, best_params['model__max_depth'], best_params['model__max_depth'] + 2],
    'model__min_samples_split': [best_params['model__min_samples_split'] - 1, best_params['model__min_samples_split'], best_params['model__min_samples_split'] + 1],
    'model__min_samples_leaf': [best_params['model__min_samples_leaf'] - 1, best_params['model__min_samples_leaf'], best_params['model__min_samples_leaf'] + 1],
    'model__max_features': [best_params['model__max_features']]
}

# Initialize Grid Search
grid_search = GridSearchCV(
    pipeline, 
    param_grid=param_grid, 
    cv=5, 
    verbose=2, 
    n_jobs=-1
)



In [20]:
# Run Grid Search
grid_search.fit(x, y)


Fitting 5 folds for each of 81 candidates, totalling 405 fits


GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('preprocessor',
                                        Pipeline(steps=[('feat_selection',
                                                         SelectPercentile(percentile=60)),
                                                        ('scaler',
                                                         MinMaxScaler())])),
                                       ('model',
                                        RandomForestRegressor(n_jobs=-1))]),
             n_jobs=-1,
             param_grid={'model__max_depth': [8, 10, 12],
                         'model__max_features': ['sqrt'],
                         'model__min_samples_leaf': [2, 3, 4],
                         'model__min_samples_split': [5, 6, 7],
                         'model__n_estimators': [190, 200, 210]},
             verbose=2)

In [22]:

# Retrieve the best parameters
best_params_grid = grid_search.best_params_


best_params = grid_search.best_params_
best_score = grid_search.best_score_
print(f"Best parameters: {best_params_grid}")
print(f"Best score: {best_score}")


Best parameters: {'model__max_depth': 10, 'model__max_features': 'sqrt', 'model__min_samples_leaf': 3, 'model__min_samples_split': 7, 'model__n_estimators': 200}
Best score: 0.6249414587238398
[CV] END model__max_depth=8, model__max_features=sqrt, model__min_samples_leaf=2, model__min_samples_split=5, model__n_estimators=210; total time=   1.9s
[CV] END model__max_depth=8, model__max_features=sqrt, model__min_samples_leaf=2, model__min_samples_split=7, model__n_estimators=190; total time=   2.3s
[CV] END model__max_depth=8, model__max_features=sqrt, model__min_samples_leaf=3, model__min_samples_split=5, model__n_estimators=200; total time=   1.9s
[CV] END model__max_depth=8, model__max_features=sqrt, model__min_samples_leaf=3, model__min_samples_split=6, model__n_estimators=200; total time=   1.8s
[CV] END model__max_depth=8, model__max_features=sqrt, model__min_samples_leaf=3, model__min_samples_split=7, model__n_estimators=200; total time=   1.9s
[CV] END model__max_depth=8, model__m